In [1]:
import heapq

def heuristic(a, b):
    """
    Calculates the Manhattan distance between two (row, col) tuples.
    This is our heuristic h(n).
    """
    (r1, c1) = a
    (r2, c2) = b
    return abs(r1 - r2) + abs(c1 - c2)

def a_star_grid_search(grid, start, goal):
    """
    Finds the shortest path from start to goal in a grid using A* search.
    
    Args:
        grid (list of lists): 2D list where 0 is open path and 1 is a wall.
        start (tuple): (row, col) of the starting position.
        goal (tuple): (row, col) of the goal position.
        
    Returns:
        (list, int): A tuple of (path, total_cost). Returns (None, 0) if no path is found.
    """
    
    rows = len(grid)
    cols = len(grid[0])
    
    # Check if start or goal is a wall
    if grid[start[0]][start[1]] == 1 or grid[goal[0]][goal[1]] == 1:
        return None, 0

    # The priority queue stores tuples: (f_cost, g_cost, current_node, path_list)
    # f_cost = g(n) + h(n)
    g_cost_start = 0
    h_cost_start = heuristic(start, goal)
    f_cost_start = g_cost_start + h_cost_start
    
    open_set = [(f_cost_start, g_cost_start, start, [start])]
    
    # g_costs stores the cheapest g_cost found *so far* to reach a node (row, col).
    # This also acts as our 'visited' or 'closed' set.
    g_costs = {start: 0}

    while open_set:
        # 1. Pop the node with the lowest f_cost
        f_cost, g_cost, current_node, path = heapq.heappop(open_set)
        
        # 2. Check if we've reached the goal
        if current_node == goal:
            return path, g_cost  # We found the shortest path!

        # 3. Explore neighbors (up, down, left, right)
        # (dr, dc) = "delta row", "delta col"
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            r, c = current_node
            neighbor = (r + dr, c + dc)
            nr, nc = neighbor  # new_row, new_col

            # --- Check if neighbor is valid ---
            # 1. Is it within the grid boundaries?
            if not (0 <= nr < rows and 0 <= nc < cols):
                continue
            # 2. Is it a wall?
            if grid[nr][nc] == 1:
                continue
            # ------------------------------------
            
            # The cost to move to a neighbor is always 1
            new_g_cost = g_cost + 1
            
            # 4. Check if this is the first time we've seen this neighbor
            #    OR if we've found a *shorter path* to it than before.
            if neighbor not in g_costs or new_g_cost < g_costs[neighbor]:
                
                # Update the g_cost for this neighbor
                g_costs[neighbor] = new_g_cost
                
                # Calculate the new f_cost
                h_cost = heuristic(neighbor, goal)
                new_f_cost = new_g_cost + h_cost
                
                # 5. Push this new, better path onto the priority queue
                new_path = path + [neighbor]
                heapq.heappush(open_set, (new_f_cost, new_g_cost, neighbor, new_path))
                
    # If the loop finishes without finding the goal
    return None, 0

# --- Main part of the script ---

# 0 = open path
# 1 = wall
grid_map = [
    [0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start_pos = (0, 0)
goal_pos = (4, 4)

# --- Run the search ---
path, cost = a_star_grid_search(grid_map, start_pos, goal_pos)

# --- Print the results ---
print(f"A* Search from {start_pos} to {goal_pos}:")
if path:
    print(f"  Path: {path}")
    print(f"  Total Cost (steps): {cost}")
    
    # Create a visual representation of the grid
    print("\nVisual Path:")
    
    # Create a copy of the grid to draw the path on
    path_grid = [row[:] for row in grid_map]

    # Draw the path
    for r, c in path:
        if (r, c) == start_pos:
            path_grid[r][c] = 'S'  # Start
        elif (r, c) == goal_pos:
            path_grid[r][c] = 'G'  # Goal
        else:
            path_grid[r][c] = '*'  # Path
            
    # Draw walls and open spaces
    for r in range(len(path_grid)):
        for c in range(len(path_grid[0])):
            if path_grid[r][c] == 1:
                path_grid[r][c] = '■'  # Wall
            elif path_grid[r][c] == 0:
                path_grid[r][c] = '.'  # Open space
    
    # Print the final grid
    for row in path_grid:
        print(" ".join(row))
        
else:
    print("  No path found.")

A* Search from (0, 0) to (4, 4):
  Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
  Total Cost (steps): 8

Visual Path:
S * * * *
■ ■ . ■ *
. . . ■ *
. ■ ■ ■ *
. . . . G
